In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE236924"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE236924"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE236924.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE236924.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE236924.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"SIRPa agonist antibody treatment ameliorates experimental arthritis and colitis [array]"
!Series_summary	"The innate immune system is finely tuned to enable. rapid response to pathogenic stimuli but keep quiescent during tissue homeostasis. Balance of activating and inhibitory signaling sets a threshold for immune activation. Signal regulatory protein (SIRPa) is an immune inhibitory receptor expressed by myeloid cells and interacts with CD47 to inhibit immune cell phagocytosis, migration, and activation. Despite the progress of SIRPa and CD47 antagonist antibodies to promote anti-cancer immunity, it is not yet known whether therapeutic SIRPa receptor agonism could restrain excessive autoimmune inflammation in the context of autoimmunity. Here, we reported that increased neutrophil- and monocyte-associated genes including SIRPA in inflamed tissues biopsies of rheumatoid arthritis and inflammatory bowel diseases, and elevated SIRPA in colonic biopsi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# Check gene expression data availability
# Based on the background information, this study compares tissues from osteoarthritis, rheumatoid arthritis, and controls
# It mentions "array" in the title, suggesting gene expression data
is_gene_available = True

# Check trait data availability from sample characteristics
# Row 0 contains disease information including 'OA' (Osteoarthritis)
trait_row = 0

# Define trait conversion function for Osteoarthritis
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary (1 for OA, 0 for non-OA)
    if "OA" in value:
        return 1
    elif "Control" in value:
        return 0
    # RA is not our trait of interest
    elif "RA" in value:
        return 0
    else:
        return None

# Age and gender information not available in sample characteristics
age_row = None
gender_row = None

def convert_age(value):
    # Not used but defined for completeness
    if pd.isna(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    # Not used but defined for completeness
    if pd.isna(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value in ["female", "f"]:
        return 0
    elif value in ["male", "m"]:
        return 1
    else:
        return None

# Initial validation and save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Clinical feature extraction if trait_row is not None
if trait_row is not None:
    # Assume clinical_data exists and was loaded in a previous step
    try:
        # Make directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Extract clinical features
        clinical_df = geo_select_clinical_features(
            clinical_data,  # This should be defined in a previous step
            trait=trait, 
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(clinical_df)
        print("Clinical data preview:", preview)
        
        # Save to CSV
        clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except:
        print("Error: clinical_data not found or other error occurred.")
        print("Make sure clinical_data is loaded before running this code.")


Clinical data preview: {'GSM7585682': [1.0], 'GSM7585683': [1.0], 'GSM7585684': [1.0], 'GSM7585685': [0.0], 'GSM7585686': [0.0], 'GSM7585687': [1.0], 'GSM7585688': [1.0], 'GSM7585689': [1.0], 'GSM7585690': [0.0], 'GSM7585691': [0.0], 'GSM7585692': [1.0], 'GSM7585693': [0.0], 'GSM7585694': [1.0], 'GSM7585695': [1.0], 'GSM7585696': [1.0], 'GSM7585697': [0.0], 'GSM7585698': [0.0], 'GSM7585699': [1.0], 'GSM7585700': [1.0], 'GSM7585701': [0.0], 'GSM7585702': [0.0], 'GSM7585703': [1.0], 'GSM7585704': [1.0], 'GSM7585705': [0.0], 'GSM7585706': [1.0], 'GSM7585707': [1.0], 'GSM7585708': [0.0], 'GSM7585709': [0.0], 'GSM7585710': [1.0], 'GSM7585711': [1.0], 'GSM7585712': [1.0], 'GSM7585713': [0.0], 'GSM7585714': [1.0], 'GSM7585715': [1.0], 'GSM7585716': [0.0], 'GSM7585717': [1.0], 'GSM7585718': [1.0], 'GSM7585719': [1.0], 'GSM7585720': [1.0], 'GSM7585721': [1.0], 'GSM7585722': [0.0], 'GSM7585723': [0.0], 'GSM7585724': [1.0], 'GSM7585725': [1.0], 'GSM7585726': [1.0], 'GSM7585727': [1.0], 'GSM758572

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE236924/GSE236924_series_matrix.txt.gz


Gene data shape: (54675, 132)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers from the output
# The identifiers like '1007_s_at', '1053_at', etc. are probe IDs from Affymetrix microarray platforms
# These are not human gene symbols but probe identifiers that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for probe IDs and gene symbols in the gene annotation data
# From the preview, we can see that 'ID' contains the probe identifiers (like '1007_s_at')
# and 'Gene Symbol' contains the gene symbols (like 'DDR1 /// MIR4640')
prob_col = 'ID'
gene_col = 'Gene Symbol'

print(f"Mapping probe IDs from column '{prob_col}' to gene symbols from column '{gene_col}'")

# 2. Get the gene mapping dataframe using the library function
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
print("Converting probe-level measurements to gene-level expression...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# 4. Normalize gene symbols to handle synonyms and aggregate rows with the same normalized symbol
print("\nNormalizing gene symbols and aggregating duplicate entries...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# 5. Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping probe IDs from column 'ID' to gene symbols from column 'Gene Symbol'


Gene mapping dataframe shape: (45782, 2)
First few rows of mapping dataframe:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Converting probe-level measurements to gene-level expression...


Gene expression data shape after mapping: (21278, 132)
First few gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')

Normalizing gene symbols and aggregating duplicate entries...
Gene expression data shape after normalization: (19845, 132)
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE236924.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (19845, 132)
Gene data column names (sample IDs):
Index(['GSM7585682', 'GSM7585683', 'GSM7585684', 'GSM7585685', 'GSM7585686'], dtype='object')



Raw clinical data structure:
Clinical data shape: (1, 133)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM7585682', 'GSM7585683', 'GSM7585684',
       'GSM7585685'],
      dtype='object')

Sample characteristics dictionary:
{0: ['disease: OA', 'disease: Control', 'disease: RA']}

Values in trait row:
['!Sample_characteristics_ch1' 'disease: OA' 'disease: OA' 'disease: OA'
 'disease: Control']

Created clinical features dataframe:
Shape: (1, 132)
               GSM7585682 GSM7585683 GSM7585684 GSM7585685 GSM7585686
Osteoarthritis          1          1          1          0          0

Linked data shape before handling missing values: (132, 19846)
Actual trait column in linked data: Osteoarthritis


/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (132, 19846)
For the feature 'Osteoarthritis', the least common label is '0' with 43 occurrences. This represents 32.58% of the dataset.
The distribution of the feature 'Osteoarthritis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Osteoarthritis/GSE236924.csv
